Muat Dataset dan Lakukan Analisis Awal 

In [3]:
# Untuk manipulasi data
import pandas as pd
import numpy as np

# Untuk Preprocessing dan Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

# Untuk Model
from sklearn.ensemble import RandomForestClassifier

# Untuk Evaluasi
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# Untuk Visualisasi
import matplotlib.pyplot as plt

# Untuk menyimpan dan memuat model
import skops.io as sio

ModuleNotFoundError: No module named 'pandas'

In [ ]:
drug_df = pd.read_csv("Data/drug200.csv")
drug_df = drug_df.sample(frac=1) # Mengacak data
drug_df.head(3)

ModuleNotFoundError: No module named 'pandas'

Pisahkan Data Latih dan Data Uji

In [ ]:
from sklearn.model_selection import train_test_split

X = drug_df.drop("Drug", axis=1).values
y = drug_df.Drug.values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=125
)

Bangun Machine Learning Pipeline 

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

# Tentukan kolom kategorikal dan numerik
cat_col = [1, 2, 3]
num_col = [0, 4]

# Buat transformer untuk preprocessing
transform = ColumnTransformer([
    ("num_imputer", SimpleImputer(strategy="median"), num_col),
    ("encoder", OrdinalEncoder(), cat_col),
    ("num_scaler", StandardScaler(), num_col),
])

# Buat pipeline utama
pipe = Pipeline(
    steps=[
        ("preprocessing", transform),
        ("model", RandomForestClassifier(n_estimators=100, random_state=125)),
    ]
)

# Latih pipeline
pipe.fit(X_train, y_train)

Evaluasi Model 

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

predictions = pipe.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average="macro")

print("Accuracy:", str(round(accuracy, 2) * 100) + "%", "F1:", round(f1, 2))

Buat dan Simpan Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

cm = confusion_matrix(y_test, predictions, labels=pipe.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipe.classes_)
disp.plot()
plt.savefig("Results/model_results.png", dpi=120)

Simpan Model Terlatih Menggunakan skops 

In [ ]:
import skops.io as sio

sio.dump(pipe, "Model/drug_pipeline.skops")